In [1]:
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName('dt').getOrCreate()

In [2]:
df = spark.sql('select * from dog_food')

In [3]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
summary| A| B| C| D| Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
 count| 490| 490| 490| 490| 490|
 mean| 5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
 stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
 min| 1| 1| 5.0| 1| 0.0|
 max| 10| 10| 14.0| 10| 1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+

In [4]:
df.groupBy('Spoiled').count().show()

+-------+-----+
Spoiled|count|
+-------+-----+
 0.0| 350|
 1.0| 140|
+-------+-----+

In [5]:
df.show(5)

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
+---+---+----+---+-------+
only showing top 5 rows

In [6]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol='features')
output = assembler.transform(df)
final_data = output.select('features','Spoiled')

In [7]:
train,test = final_data.randomSplit([0.7,0.3])

In [8]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [9]:
# GridSearch CV
rfc = RandomForestClassifier(featuresCol='features', labelCol='Spoiled', predictionCol='prediction')
grid = ParamGridBuilder().addGrid(rfc.numTrees, [100, 150,200]).build()
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Spoiled', metricName='f1')
cv = CrossValidator(estimator=rfc, estimatorParamMaps=grid, evaluator=evaluator,numFolds=5, seed=100,parallelism=2)

In [10]:
rfc_cv_model = cv.fit(train)

In [11]:
pred = rfc_cv_model.transform(test)

In [12]:
pred.show(5)

+-------------------+-------+--------------------+--------------------+----------+
 features|Spoiled| rawPrediction| probability|prediction|
+-------------------+-------+--------------------+--------------------+----------+
 [1.0,1.0,12.0,2.0]| 1.0|[11.7499999999999...|[0.07833333333333...| 1.0|
 [1.0,1.0,13.0,3.0]| 1.0|[10.2016129032258...|[0.06801075268817...| 1.0|
 [1.0,3.0,8.0,5.0]| 0.0|[149.766292266292...|[0.99844194844194...| 0.0|
 [1.0,4.0,8.0,1.0]| 0.0|[148.132128295241...|[0.98754752196827...| 0.0|
[1.0,4.0,13.0,10.0]| 1.0|[4.08233965672990...|[0.02721559771153...| 1.0|
+-------------------+-------+--------------------+--------------------+----------+
only showing top 5 rows

In [13]:
eval_f1 = evaluator.evaluate(pred)
eval_f1

Out[ 41 ]: 0.9808295728720737

In [14]:
# Feature Importance could not be found in cross validated model.Lets try without cross validation  to find feature importance
rfc = RandomForestClassifier(featuresCol='features', labelCol='Spoiled', predictionCol='prediction',numTrees=150)
model_rfc = rfc.fit(train)

In [15]:
model_rfc.featureImportances

Out[ 47 ]: SparseVector(4, {0: 0.032, 1: 0.0304, 2: 0.9049, 3: 0.0326})

In [16]:
# Feature importance for chemical 'C' is high.Thus 'C' is spoiling the dog food